In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Working with Parallel Function Calls and Mutiple Function Responses in Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ffunction-calling%2Fparallel_function_calling.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/function-calling/parallel_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini lets you create a description of a function in your code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

In this tutorial, you'll learn how to work with parallel function calling within Gemini Function Calling, including:
    
- Handling parallel function calls for repeated functions
- Working with parallel function calls across multiple functions
- Extracting multiple function calls from a Gemini response
- Calling multiple functions and returning them to Gemini

### What is parallel function calling?

In previous versions of the Gemini Pro models (prior to May 2024), Gemini would return two or more chained function calls if the model determined that more than one function call was needed before returning a natural language summary. Here, a chained function call means that you get the first function call response, return the API data to Gemini, get a second function call response, return the API data to Gemini, and so on.

In recent versions of specific Gemini Pro models (from May 2024 and on), Gemini has the ability to return two or more function calls in parallel (i.e., two or more function call responses within the first function call response object). Parallel function calling allows you to fan out and parallelize your API calls or other actions that you perform in your application code, so you don't have to work through each function call response and return one-by-one! Refer to the [Gemini Function Calling documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling) for more information on which Gemini model versions support parallel function calling.


<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png" width="80%">

## Getting Started


### Install Vertex AI SDK and other required packages

In [1]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform google-play-scraper

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>



### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Code Examples

### Import libraries

In [5]:
import google_play_scraper
from IPython.display import display, Markdown
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    Part,
    Tool,
)

### Define helper function

In [6]:
# Helper function to extract one or more function calls from a Gemini Function Call response
def extract_function_calls(response):
    function_calls = []
    for part in response.candidates[0].content.parts:
        if part.function_call:
            function_call_dict = {part.function_call.name: {}}
            for key, value in part.function_call.args.items():
                function_call_dict[part.function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls

## Example: Parallel function calls on the same function

One great use case for parallel function calling is when you have a function that only accepts one parameter per API call.

With Gemini Function Calling, rather than having to send N number of API requests to Gemini for N number of parameters and external API calls, you can send a single API request to Gemini, receive N number of Function Call Responses within a single response, make N number of external API calls, then return all of the API responses to Gemini in bulk. And you can do all of this without any extra configuration in your function declarations, tools, or requests to Gemini.

In this example, you'll do exactly that and use Parallel Function Calling in Gemini to ask about multiple app categories in the [Google Play Store](https://play.google.com/store/apps). Let's get started!

### Write function declarations and wrap them in a tool

First, define your function declarations and tool using the Gemini SDK for Python:

In [7]:
search_google_play_store = FunctionDeclaration(
    name="search_google_play_store",
    description="Search for apps in the Google Play Store",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for in the Google Play Store",
            },
        },
    },
)

google_play_store_tool = Tool(
    function_declarations=[
        search_google_play_store,
    ],
)

### Initialize the Gemini model

Now you can initialize Gemini using a [model version that supports parallel function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling):

In [8]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0514",
    generation_config=GenerationConfig(temperature=0),
    tools=[google_play_store_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

Send a prompt to Gemini that includes a phrase that you expect to invoke two or more function calls.

In this case we'll ask about three different app categories to search for in the Google Play Store:

In [9]:
prompt = """
Search for apps related to calendars and scheduling, apps for learning new languages,
and games related to word puzzles and provide a summary of your findings
"""

response = chat.send_message(prompt)

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that Gemini responded with:

In [10]:
function_calls = extract_function_calls(response)
function_calls

[{'search_google_play_store': {'query': 'calendar scheduling'}},
 {'search_google_play_store': {'query': 'learn new languages'}},
 {'search_google_play_store': {'query': 'word puzzle games'}}]

Note that the helper function is just one way to extract fields from the structured Function Call response. You can modify the helper function or write your own custom code to extract and get the fields into your preferred format or data structure!

### Make external API calls

Next, you'll loop through the Function Calls and use the `google_play_scraper` to make an API call for each search query:

In [11]:
api_response = []

# Loop through multiple function calls
for function_call in function_calls:
    print(function_call)

    # Make external API call
    result = google_play_scraper.search(
        function_call["search_google_play_store"]["query"],
        n_hits=10,
    )

    # Collect all API calls
    api_response.append(str(result))

{'search_google_play_store': {'query': 'calendar scheduling'}}
{'search_google_play_store': {'query': 'learn new languages'}}
{'search_google_play_store': {'query': 'word puzzle games'}}


### Get a natural language summary

Now you can return all of the API responses to Gemini so that it can generate a natural language summary:

In [12]:
# Return the API response to Gemini
response = chat.send_message(
    Part.from_function_response(
        name="search_google_play_store",
        response={
            "content": api_response,
        },
    ),
)
display(Markdown("<font style='color:blue'>" + response.text))

<font style='color:blue'>I found several highly-rated apps in each category:

**Calendars and Scheduling:**

* **Google Calendar:** A popular choice with a variety of views, automatic event integration from Gmail, and task management features. 
* **Business Calendar 2:** Offers a comprehensive suite of tools for planning events and tasks, syncing with various calendar services, and customizable widgets.
* **DigiCal Calendar Agenda:** Known for its clean design and customization options, including various agenda views, widgets, and global weather forecasts.
* **TimeTree - Shared Calendar:** Focuses on shared calendars for families, couples, or work, with features like notifications, memos, and in-event chat.

**Learning New Languages:**

* **Duolingo:** A gamified approach to language learning with bite-sized lessons, progress tracking, and a vast user community.
* **Babbel:** Offers tailored courses designed by language experts, focusing on real-life conversations and pronunciation improvement.
* **Busuu:** Emphasizes community interaction with native speakers for language exchange and feedback, along with structured courses and revision tools.
* **Rosetta Stone:** Utilizes its immersive Dynamic Immersion method, focusing on contextual learning and speaking confidence.
* **Memrise:** Uses real-life scenarios, native speaker videos, and AI conversation practice to build vocabulary and fluency.

**Word Puzzle Games:**

* **Word Search - Word Puzzle Game:** A classic word search game with thousands of levels, various themes, and a relax mode for casual play.
* **Wordscapes:** Combines word searching, anagrams, and crosswords with beautiful scenery backgrounds for a relaxing yet challenging experience.
* **Word Search Explorer:** Offers a quest-based word search experience with hundreds of puzzles, vocabulary-building features, and offline play.
* **Word Connect:** Features a vast dictionary, multiple game modes, daily bonuses, and a vintage style.
* **Words of Wonders:**  A crossword game that takes you on a journey around the world, improving vocabulary and spelling skills. 


And you're done with no extra configuration necessary!

Note that Gemini will use the information in your `FunctionDeclarations` to determine if and when it should return parallel Function Call responses, or it will determine which Function Calls need to be made before others to get information that a subsequent Function Call depends on. So be sure to prepare for this behavior in your logic and application code!

## Example: Parallel function calls across multiple functions

Another good fit for parallel function calling is when you have multiple, independent functions that you want to be able to call in parallel, which reduces the number of consecutive Gemini API calls that you need to make and ideally reduces the overall response time for the end user who is waiting for a natural language response.

In this example, you'll use Parallel Function Calling in Gemini to ask about multiple aspects of an app in the [Google Play Store](https://play.google.com/store/apps).

### Write function declarations and wrap them in a tool

First, define your function declarations and tool using the Gemini SDK for Python:

In [13]:
get_app_info = FunctionDeclaration(
    name="get_app_info",
    description="Get detailed information about an app in the Google Play Store",
    parameters={
        "type": "object",
        "properties": {
            "appId": {
                "type": "string",
                "description": "App ID to get detailed information about",
            },
        },
    },
)

get_app_reviews = FunctionDeclaration(
    name="get_app_reviews",
    description="Get reviews for an app in the Google Play Store",
    parameters={
        "type": "object",
        "properties": {
            "appId": {
                "type": "string",
                "description": "App ID to get reviews for",
            },
        },
    },
)

get_app_permissions = FunctionDeclaration(
    name="get_app_permissions",
    description="Get permissions for an app in the Google Play Store",
    parameters={
        "type": "object",
        "properties": {
            "appId": {
                "type": "string",
                "description": "App ID to get permissions for",
            },
        },
    },
)

google_play_store_tool = Tool(
    function_declarations=[
        get_app_info,
        get_app_reviews,
        get_app_permissions,
    ],
)

### Initialize the Gemini model

Now you can initialize Gemini using a [model version that supports parallel function calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling):

In [14]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0514",
    generation_config=GenerationConfig(temperature=0),
    tools=[google_play_store_tool],
)
chat = model.start_chat()

### Send prompt to Gemini

Send a prompt to Gemini that includes a phrase that you expect to invoke two or more function calls.

In this case we'll ask about three different types of detail to look up for a given app ID in the Google Play Store:

In [15]:
prompt = """
Show detailed app information, review sentiment, and permissions about the Google Chrome app with the app ID: com.android.chrome
"""

response = chat.send_message(prompt)

### Extract function names and parameters

Use the helper function that we created earlier to extract the function names and function parameters for each Function Call that Gemini responded with:

In [16]:
function_calls = extract_function_calls(response)
function_calls

[{'get_app_info': {'appId': 'com.android.chrome'}},
 {'get_app_reviews': {'appId': 'com.android.chrome'}},
 {'get_app_permissions': {'appId': 'com.android.chrome'}}]

### Make external API calls

Next, you'll loop through the Function Calls and use the `google_play_scraper` to make an API call to retrieve different types of information for the app:

In [17]:
api_response = []
# Loop through multiple function calls

for function_call in function_calls:
    print(function_call)

    # Extract the function name
    for key in function_call:
        function_name = key

    # Determine which external API call to make
    if function_name == "get_app_info":
        result = google_play_scraper.app(
            function_call["get_app_info"]["appId"],
        )

    if function_name == "get_app_reviews":
        result, _ = google_play_scraper.reviews(
            function_call["get_app_reviews"]["appId"],
            sort=google_play_scraper.Sort.RATING,
            count=5,
        )

    if function_name == "get_app_permissions":
        result = google_play_scraper.permissions(
            function_call["get_app_permissions"]["appId"],
        )

    # Collect all API calls
    api_response.append(str(result))

{'get_app_info': {'appId': 'com.android.chrome'}}
{'get_app_reviews': {'appId': 'com.android.chrome'}}
{'get_app_permissions': {'appId': 'com.android.chrome'}}


### Get a natural language summary

Now you can return all of the API responses to Gemini so that it can generate a natural language summary:

In [18]:
# Return the API response to Gemini
response = chat.send_message(
    Part.from_function_response(
        name="get_app_info",
        response={
            "content": api_response,
        },
    ),
)

display(Markdown("<font style='color:blue'>" + response.text))

<font style='color:blue'>Here is some information about the Google Chrome app:

**Detailed App Information:**

* Google Chrome is a fast, easy-to-use, and secure web browser. 
* It offers personalized news articles, quick links to favorite sites, downloads, and Google Search and Google Translate built-in.
* Users can browse fast, use Incognito mode, access Chrome across devices, and protect their phone with Google Safe Browsing.

**Review Sentiment:**

* Recent reviews are generally positive, praising Chrome's speed, simplicity, and reliability. 
* Some users have reported issues with recent updates, such as problems with the password manager and page refreshing. 

**Permissions:**

Chrome requests access to a wide range of permissions, including:

* **Device & app history:** Read your web bookmarks and history
* **Identity:** Add or remove accounts, find accounts on the device
* **Contacts:** Find accounts on the device, read your contacts
* **Location:** Approximate and precise location
* **Phone:** Read phone status and identity
* **Photos/Media/Files:** Modify or delete the contents of your USB storage, read the contents of your USB storage
* **Camera:** Take pictures and videos
* **Microphone:** Record audio
* **Wi-Fi connection information:** View Wi-Fi connections
* **Other:** Access Bluetooth settings, change network connectivity, change your audio settings, control Near Field Communication, control vibration, full network access, install shortcuts, pair with Bluetooth devices, prevent device from sleeping, read Google service configuration, read sync settings, reorder running apps, run at startup, toggle sync on and off, use accounts on the device, view network connections, write web bookmarks and history 

Let me know if you have any other questions about the Google Chrome app. 


And you're done! You successfully made parallel function calls for a couple of different use cases. Feel free to adapt the code samples here for your own use cases and applications! Or try another notebook to continue exploring other functionality in the Gemini API!

Happy parallel function calling!